In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import pandas as pd
import os

In [2]:
os.chdir(r'E:\Kaggle\home-data-for-ml-course')

In [3]:
housing_df=pd.read_csv('train.csv', index_col='Id')
features=['OverallQual', 'GrLivArea' ,'TotalBsmtSF' ,'BsmtFinSF1','GarageCars','1stFlrSF','GarageArea','LotArea','MasVnrArea','YearBuilt',
        '2ndFlrSF','LotFrontage','TotRmsAbvGrd','YearRemodAdd','BsmtUnfSF','WoodDeckSF','MoSold','OpenPorchSF','OverallCond',
          'Neighborhood'] #, 'GarageYrBlt']
          #'FullBath',
         #'Fireplaces']   
#Bsmt_Qual_ex,'LotShape_Reg','CentralAir_Y',,'BsmtQual_Gd' is a feature created by one hot encoding. it will be added back after preprocessing
X=housing_df[features]
#X=housing_df.drop(['SalePrice'], axis=1)
y=housing_df.SalePrice

In [4]:
X_train,X_val,y_train,y_val=train_test_split(X,y,train_size=0.75,test_size=0.25,random_state=0)

In [5]:
housing_df.shape

(1460, 80)

PRE-PROCESSING

In [7]:
X_train_num=X_train.select_dtypes(exclude=['object'])
X_val_num=X_val.select_dtypes(exclude=['object'])
X_train_obj=X_train.select_dtypes(include=['object'])
X_val_obj=X_val.select_dtypes(include=['object'])

In [8]:
numeric_cols_train = list(X_train_num.columns)
obj_cols_train = list(X_train_obj.columns)
print(numeric_cols_train)
print(obj_cols_train)

['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', 'GarageCars', '1stFlrSF', 'GarageArea', 'LotArea', 'MasVnrArea', 'YearBuilt', '2ndFlrSF', 'LotFrontage', 'TotRmsAbvGrd', 'YearRemodAdd', 'BsmtUnfSF', 'WoodDeckSF', 'MoSold', 'OpenPorchSF', 'OverallCond']
['Neighborhood']


In [9]:
col_missing_sum= X_train_num.isnull().sum()
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
print(col_missing_sum)

OverallQual       0
GrLivArea         0
TotalBsmtSF       0
BsmtFinSF1        0
GarageCars        0
1stFlrSF          0
GarageArea        0
LotArea           0
MasVnrArea        5
YearBuilt         0
2ndFlrSF          0
LotFrontage     199
TotRmsAbvGrd      0
YearRemodAdd      0
BsmtUnfSF         0
WoodDeckSF        0
MoSold            0
OpenPorchSF       0
OverallCond       0
dtype: int64


In [10]:
def get_mae(X_train_,X_val_,y_train,y_val):
    model=RandomForestRegressor(random_state=0)
    model.fit(X_train_,y_train)
    preds=model.predict(X_val_)
    mae=mean_absolute_error(y_val,preds)
    return mae

In [11]:
'''imputed_X_train_num=pd.DataFrame(index=X_train.index)
imputed_X_val_num=pd.DataFrame(index=X_val.index)

for col, i in col_missing_sum.items():
    if col_missing_sum.any()>=0.4*1460: #formula for total rows: #more than half values are NaNs, drop coumn
        reduced_X_train_num=X_train_num.drop([col], axis=1)
        reduced_X_val_num=X_val_num.drop([col], axis=1)
        print([col],get_mae(reduced_X_train_num,reduced_X_val_num,y_train,y_val))
    else:
       ''' 
myimputer=SimpleImputer()
imputed_X_train=pd.DataFrame(myimputer.fit_transform(X_train_num),index=X_train_num.index)
imputed_X_val=pd.DataFrame(myimputer.transform(X_val_num),index=X_val_num.index)  
imputed_X_train.columns=X_train_num.columns
imputed_X_val.columns=X_train_num.columns
print(get_mae(imputed_X_train,imputed_X_val,y_train,y_val))

17763.591534246574


In [12]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder=OneHotEncoder(handle_unknown='ignore',sparse_output=False)
encoded_X_train_obj=pd.DataFrame(onehot_encoder.fit_transform(X_train_obj),
                                 columns=onehot_encoder.get_feature_names_out(X_train_obj.columns))
encoded_X_val_obj=pd.DataFrame(onehot_encoder.transform(X_val_obj),
                              columns=onehot_encoder.get_feature_names_out(X_val_obj.columns))

#reattatch indices
encoded_X_train_obj.index=X_train_obj.index
encoded_X_val_obj.index=X_val_obj.index

#concatenate obj and num cols now
X_train_combi=pd.concat([encoded_X_train_obj,imputed_X_train], axis=1)
X_val_combi=pd.concat([encoded_X_val_obj,imputed_X_val], axis=1)

#ensure col name=str
X_train_combi.columns=X_train_combi.columns.astype(str)
X_val_combi.columns=X_val_combi.columns.astype(str)

print(get_mae(X_train_combi,X_val_combi,y_train,y_val))
print(X_train_combi.shape)
print(X_val_combi.shape)
print(X_train_combi.columns.equals(X_val_combi.columns))  # should be True
print(X_train_combi.head())  # all are numerical so we will skip encoding

17748.81263013699
(1095, 44)
(365, 44)
True
      Neighborhood_Blmngtn  Neighborhood_Blueste  Neighborhood_BrDale  \
Id                                                                      
1293                   0.0                   0.0                  0.0   
1019                   0.0                   0.0                  0.0   
1214                   0.0                   0.0                  0.0   
1431                   0.0                   0.0                  0.0   
811                    0.0                   0.0                  0.0   

      Neighborhood_BrkSide  Neighborhood_ClearCr  Neighborhood_CollgCr  \
Id                                                                       
1293                   0.0                   0.0                   0.0   
1019                   0.0                   0.0                   0.0   
1214                   0.0                   0.0                   0.0   
1431                   0.0                   0.0                   0.0   


In [13]:
'''model=RandomForestRegressor(n_estimators=1000, random_state=0)
model.fit(X_train_combi,y_train)
importances = model.feature_importances_
feature_names = X_train_combi.columns
importance_df = pd.DataFrame({'feature': feature_names, 'importances': importances})
top_features = importance_df.sort_values('importances', ascending=False).head(26)
print(top_features)'''

"model=RandomForestRegressor(n_estimators=1000, random_state=0)\nmodel.fit(X_train_combi,y_train)\nimportances = model.feature_importances_\nfeature_names = X_train_combi.columns\nimportance_df = pd.DataFrame({'feature': feature_names, 'importances': importances})\ntop_features = importance_df.sort_values('importances', ascending=False).head(26)\nprint(top_features)"

MODEL SELECTION

In [15]:
'''model1=RandomForestRegressor(n_estimators=570 ,max_depth=12,random_state=0)
model2=RandomForestRegressor(n_estimators=1001,min_samples_split=2,random_state=0)
model3=RandomForestRegressor(n_estimators=1000,min_samples_split=2, random_state=0)
model_list=[model1,model2,model3]                                    #min_samples_split caused higher mae in most tried values so ignored it
                                                                         #max_depth 12 has turned out to be the best depth in numerous testing runs
for m in model_list:
    model=m
    model.fit(X_train_combi,y_train)
    preds=model.predict(X_val_combi)
    mae=mean_absolute_error(y_val,preds)
    print(mae) '''

'model1=RandomForestRegressor(n_estimators=570 ,max_depth=12,random_state=0)\nmodel2=RandomForestRegressor(n_estimators=1001,min_samples_split=2,random_state=0)\nmodel3=RandomForestRegressor(n_estimators=1000,min_samples_split=2, random_state=0)\nmodel_list=[model1,model2,model3]                                    #min_samples_split caused higher mae in most tried values so ignored it\n                                                                         #max_depth 12 has turned out to be the best depth in numerous testing runs\nfor m in model_list:\n    model=m\n    model.fit(X_train_combi,y_train)\n    preds=model.predict(X_val_combi)\n    mae=mean_absolute_error(y_val,preds)\n    print(mae) '

In [16]:
'''import numpy as np
def get_model_mae(model):
    model.fit(X_train_combi,y_train)
    preds=model.predict(X_val_combi)
    mae=mean_absolute_error(y_val,preds)
    return mae

mae_list=[]
i_list=[]
for i in range(520,570):   #decreased my ranges from (70 500 50) 1690, (400 600 20) 1680, (400 600 10) 1670, 
    model=RandomForestRegressor(n_estimators=i ,max_depth=12,random_state=0,n_jobs=-1)
    mae=get_model_mae(model)
    if mae<=16845:
        print(i, mae)
        mae_list.append(mae)
        i_list.append(i)

mae_ser=pd.Series(mae_list, index=i_list)
print('min mae:',mae_ser.idxmin(), mae_ser.min())'''

"import numpy as np\ndef get_model_mae(model):\n    model.fit(X_train_combi,y_train)\n    preds=model.predict(X_val_combi)\n    mae=mean_absolute_error(y_val,preds)\n    return mae\n\nmae_list=[]\ni_list=[]\nfor i in range(520,570):   #decreased my ranges from (70 500 50) 1690, (400 600 20) 1680, (400 600 10) 1670, \n    model=RandomForestRegressor(n_estimators=i ,max_depth=12,random_state=0,n_jobs=-1)\n    mae=get_model_mae(model)\n    if mae<=16845:\n        print(i, mae)\n        mae_list.append(mae)\n        i_list.append(i)\n\nmae_ser=pd.Series(mae_list, index=i_list)\nprint('min mae:',mae_ser.idxmin(), mae_ser.min())"

In [17]:
best_model=RandomForestRegressor(n_estimators=1363, random_state=0,n_jobs=-1)
best_model.fit(X_train_combi,y_train)
preds=best_model.predict(X_val_combi)
mae=mean_absolute_error(y_val,preds)
print(mae)                               

17472.567511231267


X_test

In [19]:
X_test_raw = pd.read_csv('test.csv', index_col='Id')

numeric_cols = [col for col in numeric_cols_train if col in X_test_raw.columns]
obj_cols = [col for col in obj_cols_train if col in X_test_raw.columns]

X_test_num = X_test_raw[numeric_cols]
X_test_obj = X_test_raw[obj_cols]
col_missing_sum= X_test_num.isnull().sum()
print(col_missing_sum)
print(X_test_raw.shape)
X_test_num = X_test_raw[myimputer.feature_names_in_]  

OverallQual       0
GrLivArea         0
TotalBsmtSF       1
BsmtFinSF1        1
GarageCars        1
1stFlrSF          0
GarageArea        1
LotArea           0
MasVnrArea       15
YearBuilt         0
2ndFlrSF          0
LotFrontage     227
TotRmsAbvGrd      0
YearRemodAdd      0
BsmtUnfSF         1
WoodDeckSF        0
MoSold            0
OpenPorchSF       0
OverallCond       0
dtype: int64
(1459, 79)


In [20]:
print("Train numeric cols used to fit imputer:", myimputer.feature_names_in_)
print("X_test_num columns right now:", list(X_test_num.columns))
print("Shape of imputed test:", myimputer.transform(X_test_num).shape)
#print("Train obj cols used to fit encoder:", onehot_encoder.feature_names_in_)
print("X_test_obj columns right now:", list(X_test_obj.columns))


Train numeric cols used to fit imputer: ['OverallQual' 'GrLivArea' 'TotalBsmtSF' 'BsmtFinSF1' 'GarageCars'
 '1stFlrSF' 'GarageArea' 'LotArea' 'MasVnrArea' 'YearBuilt' '2ndFlrSF'
 'LotFrontage' 'TotRmsAbvGrd' 'YearRemodAdd' 'BsmtUnfSF' 'WoodDeckSF'
 'MoSold' 'OpenPorchSF' 'OverallCond']
X_test_num columns right now: ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', 'GarageCars', '1stFlrSF', 'GarageArea', 'LotArea', 'MasVnrArea', 'YearBuilt', '2ndFlrSF', 'LotFrontage', 'TotRmsAbvGrd', 'YearRemodAdd', 'BsmtUnfSF', 'WoodDeckSF', 'MoSold', 'OpenPorchSF', 'OverallCond']
Shape of imputed test: (1459, 19)
X_test_obj columns right now: ['Neighborhood']


In [21]:
#since number of missing rows not greater than or equal to at least half of total rows we're gonna impute
imputed_test = pd.DataFrame(myimputer.transform(X_test_num))
imputed_test.columns=X_test_num.columns

#we will skip ohe because all the columns are numerical
#onehot encoding
combined_obj = pd.concat([X_train_obj, X_test_obj])  # not val now
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
onehot_encoder.fit(combined_obj)

# Re-transform both train and test
encoded_X_train_obj = pd.DataFrame(
    onehot_encoder.transform(X_train_obj),
    columns=onehot_encoder.get_feature_names_out(X_train_obj.columns),
    index=X_train_obj.index
)

encoded_X_test = pd.DataFrame(
    onehot_encoder.transform(X_test_obj),
    columns=onehot_encoder.get_feature_names_out(X_test_obj.columns),
    index=X_test_obj.index
)
X_test_combi = imputed_test.copy()
print("Unique Ids in test.csv:", len(X_test_raw.index.unique()))
print("Total Ids in test.csv:", len(X_test_raw.index))
print("Duplicate Ids:", X_test_raw.index.duplicated().sum())
print("X_test_combi shape before fix:", X_test_combi.shape)
print("X_test_combi shape after fix:", X_test_combi.shape)

Unique Ids in test.csv: 1459
Total Ids in test.csv: 1459
Duplicate Ids: 0
X_test_combi shape before fix: (1459, 19)
X_test_combi shape after fix: (1459, 19)


In [22]:
print(X_test_combi.head())  
print(X_train_combi.columns.equals(X_test_combi.columns)) 

print(X_test_combi.shape)

   OverallQual  GrLivArea  TotalBsmtSF  BsmtFinSF1  GarageCars  1stFlrSF  \
0          5.0      896.0        882.0       468.0         1.0     896.0   
1          6.0     1329.0       1329.0       923.0         1.0    1329.0   
2          5.0     1629.0        928.0       791.0         2.0     928.0   
3          6.0     1604.0        926.0       602.0         2.0     926.0   
4          8.0     1280.0       1280.0       263.0         2.0    1280.0   

   GarageArea  LotArea  MasVnrArea  YearBuilt  2ndFlrSF  LotFrontage  \
0       730.0  11622.0         0.0     1961.0       0.0         80.0   
1       312.0  14267.0       108.0     1958.0       0.0         81.0   
2       482.0  13830.0         0.0     1997.0     701.0         74.0   
3       470.0   9978.0        20.0     1998.0     678.0         78.0   
4       506.0   5005.0         0.0     1992.0       0.0         43.0   

   TotRmsAbvGrd  YearRemodAdd  BsmtUnfSF  WoodDeckSF  MoSold  OpenPorchSF  \
0           5.0        1961.0    

In [23]:
print("X_test_raw shape:", X_test_raw.shape)
print("X_test_combi shape:", X_test_combi.shape)

X_test_raw shape: (1459, 79)
X_test_combi shape: (1459, 19)


In [24]:
print("Fixed X_test_combi shape:", X_test_combi.shape)
print("ID count:", len(X_test_raw.index))

Fixed X_test_combi shape: (1459, 19)
ID count: 1459


In [25]:
print("Training features:", X_train.columns.tolist())
X_test_combi = X_test_combi.reindex(columns=X_test_combi.columns, fill_value=0)

Training features: ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'BsmtFinSF1', 'GarageCars', '1stFlrSF', 'GarageArea', 'LotArea', 'MasVnrArea', 'YearBuilt', '2ndFlrSF', 'LotFrontage', 'TotRmsAbvGrd', 'YearRemodAdd', 'BsmtUnfSF', 'WoodDeckSF', 'MoSold', 'OpenPorchSF', 'OverallCond', 'Neighborhood']


In [26]:
print("Any NaNs?    ", X_test_combi.isna().values.any())
print("All-zero col:", (X_test_combi.std(axis=0) == 0).sum(), "of", X_test_combi.shape[1])
print("Row sample:")
display(X_test_combi.head())

Any NaNs?     False
All-zero col: 0 of 19
Row sample:


,OverallQual,GrLivArea,TotalBsmtSF,BsmtFinSF1,GarageCars,1stFlrSF,GarageArea,LotArea,MasVnrArea,YearBuilt,2ndFlrSF,LotFrontage,TotRmsAbvGrd,YearRemodAdd,BsmtUnfSF,WoodDeckSF,MoSold,OpenPorchSF,OverallCond
0,5.0,896.0,882.0,468.0,1.0,896.0,730.0,11622.0,0.0,1961.0,0.0,80.0,5.0,1961.0,270.0,140.0,6.0,0.0,6.0
1,6.0,1329.0,1329.0,923.0,1.0,1329.0,312.0,14267.0,108.0,1958.0,0.0,81.0,6.0,1958.0,406.0,393.0,6.0,36.0,6.0
2,5.0,1629.0,928.0,791.0,2.0,928.0,482.0,13830.0,0.0,1997.0,701.0,74.0,6.0,1998.0,137.0,212.0,3.0,34.0,5.0
3,6.0,1604.0,926.0,602.0,2.0,926.0,470.0,9978.0,20.0,1998.0,678.0,78.0,7.0,1998.0,324.0,360.0,6.0,36.0,6.0
4,8.0,1280.0,1280.0,263.0,2.0,1280.0,506.0,5005.0,0.0,1992.0,0.0,43.0,5.0,1992.0,1017.0,0.0,1.0,82.0,5.0


In [27]:
print(X_train_combi.columns,X_val_combi.columns)
X_test_combi = X_test_combi.reindex(columns=best_model.feature_names_in_,
                                    fill_value=0)

Index(['Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale',
       'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
       'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber',
       'Neighborhood_Veenker', 'OverallQual', 'GrLivArea', 'TotalBsmtSF',
       'BsmtFinSF1', 'GarageCars', '1stFlrSF', 'GarageArea', 'LotArea',
       'MasVnrArea', 'YearBuilt', '2ndFlrSF', 'LotFrontage', 'TotRmsAbvGrd',
       'YearRemodAdd', 'BsmtUnfSF', 'WoodDeckSF', 'MoSold', 'OpenPorchSF',
       'OverallCond'],
      dtype='object') Index(['Neighborhood_Blmngtn', 'Ne

In [28]:
prediction=best_model.predict(X_test_combi)

In [29]:
print("Prediction length:", len(prediction))

Prediction length: 1459


In [30]:
print("Prediction stats:")
print("Min:", prediction.min())
print("Max:", prediction.max())
print("Mean:", prediction.mean())
print("Sample predictions:", prediction[:10])
print("SalePrice stats in training:")
print("Min:", y_train.min())
print("Max:", y_train.max())
print("Mean:", y_train.mean())

Prediction stats:
Min: 56524.70286133529
Max: 506312.95964783564
Mean: 179119.51389332383
Sample predictions: [129060.54365371 158858.04035216 183939.20689655 185544.31988261
 198085.5590609  185457.1027146  167285.22083639 176485.4380044
 186082.99779897 124400.88848129]
SalePrice stats in training:
Min: 34900
Max: 755000
Mean: 180733.1497716895


In [31]:
output = pd.DataFrame({'Id': X_test_raw.index,
                       'SalePrice': prediction})
output.to_csv('prediction1.csv', index=False)